In [1]:
import pip_system_certs  # noqa: F401  (just importing activates it)


In [4]:
from transformers import pipeline

categories = ["Fiction", "Non-Fiction"]
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

c:\Users\jeroen.vander.putten\Development\Learning\Python\github_portfolio_jeroen\project_7k_books\.venv311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jeroen.vander.putten\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [5]:
import pandas as pd

df = pd.read_csv("books_cleaned.csv")

In [6]:
category_mapping = {'Fiction' : "Fiction",
 'Juvenile Fiction': "Children's Fiction",
 'Biography & Autobiography': "Non-Fiction",
 'History': "Non-Fiction",
 'Literary Criticism': "Non-Fiction",
 'Philosophy': "Non-Fiction",
 'Religion': "Non-Fiction",
 'Comics & Graphic Novels': "Fiction",
 'Drama': "Fiction",
 'Juvenile Nonfiction': "Children's Non-Fiction",
 'Science': "Non-Fiction",
 'Poetry': "Fiction"}

df["simple_categories"] = df["categories"].map(category_mapping)

In [7]:
sequence = df.loc[df["simple_categories"] == "Fiction", "description"].reset_index(drop=True)[0]

In [8]:
import numpy as np

max_index = np.argmax(classifier(sequence, categories)["scores"])
max_label = classifier(sequence, categories)["labels"][max_index]
max_label


'Fiction'

In [10]:
def generate_predictions(sequence, categories):
    predictions = classifier(sequence, categories)
    max_index = np.argmax(predictions["scores"])
    max_label = predictions["labels"][max_index]
    return max_label

In [11]:
from tqdm import tqdm

actual_cats = []
predicted_cats = []

for i in tqdm(range(0, 300)):
    sequence = df.loc[df["simple_categories"] == "Fiction", "description"].reset_index(drop=True)[i]
    predicted_cats += [generate_predictions(sequence, categories)]
    actual_cats += ["Fiction"]

100%|██████████| 300/300 [03:27<00:00,  1.45it/s]


In [12]:
for i in tqdm(range(0, 300)):
    sequence = df.loc[df["simple_categories"] == "Non-Fiction", "description"].reset_index(drop=True)[i]
    predicted_cats += [generate_predictions(sequence, categories)]
    actual_cats += ["Non-Fiction"]

100%|██████████| 300/300 [02:42<00:00,  1.85it/s]


In [13]:
predictions_df = pd.DataFrame({"actual_categories": actual_cats, "predicted_categories": predicted_cats})
predictions_df["correct_prediction"] = (
    np.where(predictions_df["actual_categories"] == predictions_df["predicted_categories"], 1, 0)
)

In [14]:
predictions_df["correct_prediction"].sum() / len(predictions_df)

np.float64(0.665)

In [15]:
isbns = []
predicted_cats = []

missing_cats = df.loc[df["simple_categories"].isna(), ["isbn13", "description"]].reset_index(drop=True)

In [16]:
for i in tqdm(range(0, len(missing_cats))):
    sequence = missing_cats["description"][i]
    predicted_cats += [generate_predictions(sequence, categories)]
    isbns += [missing_cats["isbn13"][i]]

100%|██████████| 1454/1454 [13:53<00:00,  1.74it/s]


In [17]:
missing_predicted_df = pd.DataFrame({"isbn13": isbns, "predicted_categories": predicted_cats})

In [18]:
df = pd.merge(df, missing_predicted_df, on="isbn13", how="left")
df["simple_categories"] = np.where(df["simple_categories"].isna(), df["predicted_categories"], df["simple_categories"])
df = df.drop(columns = ["predicted_categories"])

In [19]:
df.to_csv("books_with_categories.csv", index=False)